Example Try Finally remplacé par with

In [1]:
file = open("./fichier.txt")
try:
    for line in file:
        print(line.strip())
finally:
    file.close()

Ligne 1
Ligne 2
Ligne 3
Ligne 4
Ligne 5


Remplacé par with

In [3]:
with open("./fichier.txt") as file:
    for line in file:
        print(line.strip())


Ligne 1
Ligne 2
Ligne 3
Ligne 4
Ligne 5


Faire son propre context manager pour l'utlisation avec with

In [13]:
class ContexteIllustration:
    def __enter__(self):
        print(" Je suis entrée dans le contexte, des opération de préparations")
        
    def __exit__(self, exc_type, exc_value, traceback):
        print("On quitte le contexte")
        if exc_type is None:
            print("Pas d'erreurs pendant les instructions dans le contexte de with")
            return True
        else:
            print(f"Type of the Exception : {exc_type}")
            print(f"Value of the Isntance : {exc_value}")
            return True


with ContexteIllustration():
    print( " Je suis dans le contexte de with : Instruction 1")
    print( " Je suis dans le contexte de with : Instruction 2")
    raise ValueError(" La donnée est pas bonne")
# d'exception         

 Je suis entrée dans le contexte, des opération de préparations
 Je suis dans le contexte de with : Instruction 1
 Je suis dans le contexte de with : Instruction 2
On quitte le contexte
Type of the Exception : <class 'ValueError'>
Value of the Isntance :  La donnée est pas bonne


Exemple avec application de patch temporaire

In [14]:

class Config:
    etat = "normal"

config = Config()

etat_orignal = config.etat

try: 
    config.etat = "temporaire"
    print("Instruction 1")
finally:
    config.etat = etat_orignal

Instruction 1


In [18]:
class ConfigManager:
    def __init__(self, currentConfig, etatTemporaire):
        self.currentConfig = currentConfig
        self.etatTemporaire = etatTemporaire
        
    def __enter__(self):
        self.currentEtat = self.currentConfig.etat
        self.currentConfig.etat = self.etatTemporaire
        print(f"Etat Courant : {self.currentConfig.etat}")
        print("fin __enter__")
        
    def __exit__(self, exc_type, exc_value, traceback):
        self.currentConfig.etat = self.currentEtat
        print(f"Etat Courant : {self.currentConfig.etat}")
        print("fin __exit__")
        return True
        
config = Config()

with ConfigManager(config, "temp"):
    print("instruction 1")
config.etat = "Precision10-5"
with ConfigManager(config, "temporatire_etat"):
    print("instruction 2")




Etat Courant : temp
fin __enter__
instruction 1
Etat Courant : normal
fin __exit__
Etat Courant : temporatire_etat
fin __enter__
instruction 2
Etat Courant : Precision10-5
fin __exit__


In [20]:
class ContexteIllustrationWithRetour:
    def __enter__(self):
        print(" Je suis entrée dans le contexte, des opération de préparations")
        return "Ceci est ma valeur de retour"
        
    def __exit__(self, exc_type, exc_value, traceback):
        print("On quitte le contexte")
        if exc_type is None:
            print("Pas d'erreurs pendant les instructions dans le contexte de with")
            return True
        else:
            print(f"Type of the Exception : {exc_type}")
            print(f"Value of the Isntance : {exc_value}")
            return True


with ContexteIllustrationWithRetour() as valeur_de_retour_de_enter:
    print(valeur_de_retour_de_enter)
    print( " Je suis dans le contexte de with : Instruction 1")
    print( " Je suis dans le contexte de with : Instruction 2")
# d'exception       

 Je suis entrée dans le contexte, des opération de préparations
Ceci est ma valeur de retour
 Je suis dans le contexte de with : Instruction 1
 Je suis dans le contexte de with : Instruction 2
On quitte le contexte
Pas d'erreurs pendant les instructions dans le contexte de with


Exerice FakeDB : correction

In [31]:
class FakeDB:
    def begin(self):
        print("Transaction débutée")

    def commit(self):
        print("Transaction validée (commit)")

    def rollback(self):
        print("Transaction annulée (rollback)")

class Transaction:
    def __init__(self):
        self.db = FakeDB()
        
    def __enter__(self):
        print("Je suis entrée dans le contexte.")
        self.db.begin()
        return self.db
        
    def __exit__(self, exc_type, exc_value, traceback):
        print("Nous quittons le contexte")
        if exc_type is None:
            print("Pas d'erreurs pendant les instructions dans le contexte de with")
            self.db.commit()
            return True
        else:
            print(f"Type of the Exception : {exc_type}")
            print(f"Value of the Isntance : {exc_value}")
            self.db.rollback()
            return True
            
fakeDB = FakeDB()
with Transaction() as db:
    print("instruction 1")
    raise ValueError("")

Je suis entrée dans le contexte.
Transaction débutée
instruction 1
Nous quittons le contexte
Type of the Exception : <class 'ValueError'>
Value of the Isntance : 
Transaction annulée (rollback)


Exerice 2 : Benchmark

In [34]:
import time
class PreciseTimer:
    def __init__(self, precision: int = 0):
        self.t0 = None
        self.t1 = None
        self.time = None
        self.precision = precision
 
    def __enter__(self):
        self.t0 = time.perf_counter()  # Better than time()
 
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type:
            return False
 
        self.t1 = time.perf_counter()
        self.time = self.t1 - self.t0
        print(round(self.time, self.precision))
        return True
 
 
with PreciseTimer() as timer:
    sum(range(100000000))

10.0


Démonstration de contextlib

In [37]:
class ContexteIllustration:
    def __enter__(self):
        print(" Je suis entrée dans le contexte, des opération de préparations")
        
    def __exit__(self, exc_type, exc_value, traceback):
        print("On quitte le contexte")
        if exc_type is None:
            print("Pas d'erreurs pendant les instructions dans le contexte de with")
            return True
        else:
            print(f"Type of the Exception : {exc_type}")
            print(f"Value of the Isntance : {exc_value}")
            return True

from contextlib import contextmanager

@contextmanager
def contexte_illustration():
    print(" Je suis entrée dans le contexte, des opération de préparations")
    try:
        yield #place où les instructions du contexte vont se faire
    except Exception as e:
        print(f"Type of the Exception : {type(e)}")
        print(f"Value of the Isntance : {e}")
    else:
        print("Pas d'erreurs pendant les instructions dans le contexte de with")
    finally:
        print("Instruction qui sera tout le temps executé exceptions ou pas")
  
with contexte_illustration():
    print("je suis dans le contexte")
    raise ValueError("")
        

 Je suis entrée dans le contexte, des opération de préparations
 je suis dans le contexte
Type of the Exception : <class 'ValueError'>
Value of the Isntance : 
Instruction qui sera tout le temps executé exceptions ou pas


Démonstration de contextlib :-> closing

In [38]:
class FakeDbConnection:
    def __init__(self):
        print("Ouverture de la connexion à la db")
    def close(self):
        print("Fermeture de la connexion")
"""
conn = FakeDbConnection()
try:
    instruction 1
    instruction 2
finally:
    conn.close()
"""
from contextlib import closing

with closing(FakeDbConnection()) as conn:
    print("instruction 1")

Ouverture de la connexion à la db
instruction 1
Fermeture de la connexion


Démonstration de contextlib :-> suppress

In [43]:
from contextlib import suppress
with suppress(FileNotFoundError):
    with open("./fichier_qui_nexiste_pas.txt") as file:
        print(file.read())   

In [52]:
import io
from contextlib import redirect_stdout, redirect_stderr
#io.StringIO est un buffer de string sur la RAM
with io.StringIO() as buf, redirect_stdout(buf):
    print("Ceci est normallement envoyé dans la console")
    output = buf.getvalue()
print(output)    # mettre en commentaire pur voir que le premier print n'est plus en destionation de la console 

Ceci est normallement envoyé dans la console



Double gestionnaire de contexte

In [54]:
with open("./fichier.txt") as file1, open("./fichier2.txt") as file2:
        for line1, line2 in zip(file1, file2):
            print(line1.strip() + line2.strip())

Ligne 1Ligne 1
Ligne 2Ligne 2
Ligne 3Ligne 3
Ligne 4Ligne 4
Ligne 5Ligne 5
